In [7]:
#imports

import tensorflow as tf
import os
import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Check if we have Tensorflow GPU
config=tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth=True

tf.__version__sess = tf.Session(config=config)
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[]

In [6]:
# Step 1 - Collect Data
img_width, img_height = 150, 150
train_data_dir = os.path.expanduser("images/training")
validation_data_dir = os.path.expanduser("images/validation")

In [4]:
# Rescale the input pixels
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 39801 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [5]:
# Step 2 - Build Model
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

# To prevent overfiting we use Dropout
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
# Decrease Learning Rate Every Other Epoch

from keras.callbacks import LearningRateScheduler
def scheduler(epoch):
    if epoch%2==0 and epoch!=0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr*.9)
        print("lr changed to {}".format(lr*.9))
    return K.get_value(model.optimizer.lr)

lr_decay = LearningRateScheduler(scheduler)

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
nb_epoch = 30
nb_train_samples = 38000
nb_validation_samples = 4000

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples / 16,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples / 32,
        verbose=1,
        callbacks=[lr_decay])

/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=2375.0, validation_data=<keras.pre..., validation_steps=125.0, verbose=True, epochs=30)`
  import sys


Epoch 1/30
2375/2375 [==============================] - 182s - loss: 0.6429 - acc: 0.6338 - val_loss: 0.5851 - val_acc: 0.6845
Epoch 2/30
2136/2375 [=========================>....] - ETA: 16s - loss: 0.5835 - acc: 0.6974

In [ ]:
model.save_weights('models/cnn-image-recognition2.h5')

In [ ]:
#model.load_weights('models_trained/basic_cnn_20_epochs.h5')